In [1]:
import os
import json

In [2]:
root_path = './SLTCDataset/'
datasets = ["20NGPreprocessed", "R52", "R8", 'mr', 'ohsumed_single_23']

In [3]:
with open('SLTCPreprocessConfig.json') as f:
    config = json.load(f)
assert config['Dataset'] in datasets, "Dataset is not in dataset list"